In [1]:
CELLS = [(q, r) for q in range(-3, +3 + 1) for r in range(-3, +3 + 1) if -q - r in range(-3, +3 + 1)]

In [84]:
import tensorflow as tf
import numpy as np
from keras import models, layers
from keras.models import load_model
import json
import pickle
import os
from os.path import join
import subprocess
import json
import utils

In [3]:
sequence = sorted([p for p in CELLS])

In [49]:
model = build_model()

In [107]:
a = np.array([[1]*39])
b = [500]

In [48]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(128,activation='relu', input_shape=(39,)))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [5]:
try:
    model = load_model('my_model.h5')
except:
    print("none model found")
    model = build_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [7]:
!ls '../rec'

'[1win]1556349534.0caobluejiba.txt'   '[2zzz]1556349670.0caoredjiba.txt'
'[1win]1556349557.0caobluejiba.txt'   '[2zzz]1556349953.0caobluejiba.txt'
'[1win]1556349670.0caobluejiba.txt'   '[2zzz]1556349953.0caogreenjiba.txt'
'[1win]1556349953.0caoredjiba.txt'    '[2zzz]1556350151.0caoredjiba.txt'
'[1win]1556350152.0caobluejiba.txt'   '[2zzz]1556350152.0caogreenjiba.txt'
'[1win]1556350389.0caogreenjiba.txt'  '[2zzz]1556350389.0caobluejiba.txt'
'[1win]1556352753.0caobluejiba.txt'   '[2zzz]1556350389.0caoredjiba.txt'
'[1win]1556353290.0caobluejiba.txt'   '[2zzz]1556352752.0caoredjiba.txt'
'[1win]1556353931.0caoredjiba.txt'    '[2zzz]1556352753.0caogreenjiba.txt'
'[2zzz]1556349534.0caogreenjiba.txt'  '[2zzz]1556353290.0caogreenjiba.txt'
'[2zzz]1556349534.0caoredjiba.txt'    '[2zzz]1556353290.0caoredjiba.txt'
'[2zzz]1556349557.0caogreenjiba.txt'  '[2zzz]1556353931.0caobluejiba.txt'
'[2zzz]1556349557.0caoredjiba.txt'    '[2zzz]1556353931.0caogreenjiba.txt'
'[2zzz]1556349670.0caogreenjiba.txt'


In [77]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        ls.append(line)
    return ls

In [78]:
def join_list(l):
    res = ""
    for i in l:
        res += i + "/"
    
    res = res[:-1]
    return res

In [92]:
def load_logs():
    datapath = join(join_list(os.getcwd().split('/')[:-1]),'rec/')
    files = bash_call(['ls', datapath])
    return files

In [140]:
load_logs()

['[1win]1556524801.0caobluejiba.txt',
 '[1win]1556524809.0caobluejiba.txt',
 '[1win]1556524823.0caobluejiba.txt',
 '[1win]1556524827.0caoredjiba.txt',
 '[2zzz]1556524801.0caogreenjiba.txt',
 '[2zzz]1556524801.0caoredjiba.txt',
 '[2zzz]1556524809.0caogreenjiba.txt',
 '[2zzz]1556524809.0caoredjiba.txt',
 '[2zzz]1556524823.0caogreenjiba.txt',
 '[2zzz]1556524823.0caoredjiba.txt',
 '[2zzz]1556524827.0caobluejiba.txt',
 '[2zzz]1556524827.0caogreenjiba.txt']

In [151]:
def preprocess_log(filename, condition:None):
    
    if (condition != None) and (condition not in filename):
        return False, False
    
    print(filename)
    
    states, color = utils.load_l('../rec/'+filename)

    res = []
    rewards = []

    for s in states:
        re = []
        for i in sequence:
            re.append(s[0][i])
        re.append(color)
        re.append(-s[4])
        rewards.append(s[1])
        res.append(re)

    decay = 0.97
    curr = 1
    target = [rewards[-1]]
    
    for i in range(len(res)-2, -1, -1):
        last = target[-1]
        target.append(decay*last+rewards[i])
        
    target.reverse()
    train = np.array(res)
    return res, target

In [152]:
for i in load_logs():
    res, target = preprocess_log(i, '2zzz')
    if res:
        print(len(res[-1]), len(target))
        model.fit(res, target, epochs=1, batch_size=1)

[2zzz]1556524801.0caogreenjiba.txt
39 79


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 79 arrays: [array([[2],
       [0],
       [2],
       [2],
       [0],
       [2],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       ...

In [147]:
res[-1]

[2,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 1,
 0]

In [148]:
target

[-239.8250946601956,
 -247.24236562906762,
 -254.88903673099756,
 -262.7722028154614,
 -270.8991781602695,
 -279.2775032580098,
 -287.9149518123813,
 -296.8195379509085,
 -305.9995236607304,
 -315.46342645436124,
 -325.2200272725374,
 -335.27837863148187,
 -345.6478130221463,
 -356.337951569223,
 -367.35871295796187,
 -378.7203226370741,
 -390.43332230626197,
 -402.5085796971773,
 -414.95729865688384,
 -427.7910295431792,
 -441.0216799414219,
 -454.6615257128061,
 -468.7232223843362,
 -483.21981689106826,
 -498.1647596815137,
 -513.5719171974368,
 -529.4555847396256,
 -545.8304997315728,
 -562.711855393374,
 -580.1153148385299,
 -598.0570256067319,
 -616.5536346461154,
 -635.6223037588818,
 -655.2807255246204,
 -675.5471397161035,
 -696.4403502227872,
 -717.9797424977188,
 -740.18530154404,
 -556.892063447464,
 -574.1155293272825,
 -591.871679718848,
 -403.99142239056494,
 -416.48600246449996,
 -429.36701285,
 -442.646405,
 -456.3365,
 -470.45,
 -485.0,
 -500]

In [114]:
n = model.predict(np.array([res[-1]]))
n

array([[450.5655]], dtype=float32)

In [111]:
def reload_model():
    del model  # deletes the existing model

    # returns a compiled model
    # identical to the previous one
    model = load_model('my_model.h5')


In [112]:
def save_model():
    model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    with open('trained_model', 'wb') as f:
        pickle.dump(model.get_weights(), f)

In [115]:
save_model()

In [51]:
model.predict(np.array([[1]*39,[0.5]*39]))

array([[-0.0557806],
       [-0.0278903]], dtype=float32)